In [89]:
#!pip install autogluon
#!pip install pandas

In [90]:
import pandas as pd
from autogluon.tabular import TabularPredictor
from sklearn.model_selection import TimeSeriesSplit
from sklearn.model_selection import train_test_split
from sklearn.model_selection import train_test_split
from sklearn.model_selection._split import _BaseKFold
import numpy as np
import random

In [91]:
seed_value = 42  # Replace with your desired seed value
random.seed(seed_value)
np.random.seed(seed_value)

In [92]:
x_train_c = pd.read_csv('cleaned_data_quarterly/C/x_train_c.csv')
y_train_c = pd.read_csv('cleaned_data_quarterly/C/train_c.csv')
x_test_c = pd.read_csv('cleaned_data_quarterly/C/x_test_c.csv')

In [93]:
x_train_c_combined = x_train_c.merge(y_train_c, left_on='date_forecast', right_on='time', how='left')
x_train_c_combined['observed'] = x_train_c_combined['calc_year'].isna().astype(int)
train_data_c = x_train_c_combined.drop(['time', 'calc_year', 'calc_month', 'calc_day', 'calc_hour'], axis = 1)
x_test_c['observed'] = x_test_c['calc_year'].isna().astype(int)
test_data_c = x_test_c.drop([ 'calc_year', 'calc_month', 'calc_day', 'calc_hour','date_forecast'], axis = 1)

# Trying with validation set equal to half of the estimated data


In [94]:
test_data_c['snow_density:kgm3_Q0'] = np.nan
test_data_c['snow_density:kgm3_Q1'] = np.nan
test_data_c['snow_density:kgm3_Q2'] = np.nan
test_data_c['snow_density:kgm3_Q3'] = np.nan
train_data_c.drop(columns= ['index'], inplace= True)

In [95]:

observed = train_data_c[train_data_c['observed'] == 1]
true =observed[observed['forecast_month'].isin([5, 6, 7])]
true

,date_forecast,absolute_humidity_2m:gm3_Q0,absolute_humidity_2m:gm3_Q1,absolute_humidity_2m:gm3_Q2,absolute_humidity_2m:gm3_Q3,air_density_2m:kgm3_Q0,air_density_2m:kgm3_Q1,air_density_2m:kgm3_Q2,air_density_2m:kgm3_Q3,ceiling_height_agl:m_Q0,...,forecast_year,forecast_month,forecast_day,forecast_hour,direct_rad_x_sun_elevation_Q0,direct_rad_x_sun_elevation_Q1,direct_rad_x_sun_elevation_Q2,direct_rad_x_sun_elevation_Q3,pv_measurement,observed
4516,2020-05-01 00:00:00,3.5,3.5,3.5,3.5,1.260,1.257,1.255,1.253,NaN,...,2020,5,1,0,-0.000000,-0.000,-0.000000,-0.00000,0.0,1
4517,2020-05-01 01:00:00,3.5,3.5,3.5,3.5,1.250,1.250,1.250,1.249,4823.3,...,2020,5,1,1,-0.000000,-0.000,-0.000000,-0.00000,0.0,1
4518,2020-05-01 02:00:00,3.6,3.6,3.7,3.7,1.249,1.249,1.249,1.249,5061.4,...,2020,5,1,2,-0.000000,-0.000,-0.000000,-0.00000,0.0,1
4519,2020-05-01 03:00:00,3.8,3.8,3.9,3.9,1.249,1.249,1.248,1.248,4158.7,...,2020,5,1,3,-0.000000,0.000,0.000000,0.00000,0.0,1
4520,2020-05-01 04:00:00,4.0,4.1,4.1,4.2,1.248,1.247,1.247,1.246,3666.2,...,2020,5,1,4,13.497500,34.815,74.463295,124.23259,19.6,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14317,2021-07-31 19:00:00,8.6,8.7,8.7,8.7,1.209,1.209,1.209,1.210,1061.6,...,2021,7,31,19,21.331602,0.000,0.000000,0.00000,19.6,1
14318,2021-07-31 20:00:00,8.8,8.8,8.8,8.9,1.210,1.210,1.211,1.211,985.6,...,2021,7,31,20,0.000000,-0.000,-0.000000,-0.00000,0.0,1
14319,2021-07-31 21:00:00,8.9,9.0,9.0,9.1,1.212,1.212,1.213,1.213,942.7,...,2021,7,31,21,-0.000000,-0.000,-0.000000,-0.00000,0.0,1
14320,2021-07-31 22:00:00,9.1,9.1,9.1,9.1,1.214,1.214,1.215,1.215,824.9,...,2021,7,31,22,-0.000000,-0.000,-0.000000,-0.00000,0.0,1


In [96]:
from sklearn.model_selection import train_test_split

def split_dataset(train_data, date_column='forecast_month', observed_column='observed'):
    """
    Splits the dataset into a training set and a validation set.
    The validation set includes approximately 50% of the estimated data, evenly distributed across months.
    Additionally, it includes about half of the observed data for May, June, and July, if present.
    The training set includes all months, excluding the observed data that is included in the validation set.
    
    :param train_data: The original training dataset as a pandas DataFrame.
    :param date_column: The name of the column that contains the month information.
    :param observed_column: The name of the column that indicates if the data is observed.
    :return: A tuple (training_set, validation_set)
    """
    # Work with a copy to avoid modifying the original DataFrame
    train_data = train_data.copy()
    train_data.sort_values(by='date_forecast', inplace=True)
    
    # Separate observed and estimated data
    estimated_data = train_data[train_data[observed_column] == 0]
    observed_data = train_data[train_data[observed_column] != 0]
    
    # Split the estimated data into training and validation sets
    estimated_train, estimated_val = train_test_split(
        estimated_data, test_size=0.5, random_state=42, stratify=estimated_data[date_column]
    )
    
    # Check if there are any observed data for May, June, and July
    if not observed_data[observed_data[date_column].isin([5, 6, 7])].empty:
        print("hellooo")
        observed_may_june_july = observed_data[observed_data[date_column].isin([5, 6, 7])]
        observed_train_mjj, observed_val_mjj = train_test_split(
            observed_may_june_july, test_size=0.5, random_state=42, stratify=observed_may_june_july[date_column]
        )
    else:
        
        observed_train_mjj = pd.DataFrame()
        observed_val_mjj = pd.DataFrame()
    
    # Combine the estimated and observed May, June, July data for the validation set
    validation_set = pd.concat([estimated_val, observed_val_mjj])
    validation_set.sort_values(by='date_forecast', inplace=True)
    
    # The rest of the observed data (excluding May, June, July) will be added to the training set
    observed_rest = observed_data[~observed_data[date_column].isin([5, 6, 7])]
    
    # Combine all training parts for the final training set
    training_set = pd.concat([estimated_train, observed_train_mjj, observed_rest])
    training_set.sort_values(by='date_forecast', inplace=True)
    
    return training_set, validation_set

# Create a copy of the DataFrame to pass to the function
train_data_copy = train_data_c.copy()
train_data_c, val_data_c = split_dataset(train_data_copy, 'forecast_month', 'observed')

print(len(train_data_c))
print(len(val_data_c))

hellooo
17987
3673


In [97]:
predictor_c= TabularPredictor(label ='pv_measurement',eval_metric= 'mean_absolute_error').fit(train_data = train_data_c,tuning_data = val_data_c,  use_bag_holdout=True,verbosity = 2,presets='best_quality',excluded_model_types=['CatBoost'], time_limit= 1500)

No path specified. Models will be saved in: "AutogluonModels\ag-20231107_135414\"
Presets specified: ['best_quality']
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=8, num_bag_sets=20
Beginning AutoGluon training ... Time limit = 1500s
AutoGluon will save models to "AutogluonModels\ag-20231107_135414\"
AutoGluon Version:  0.8.2
Python Version:     3.9.13
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.22621
Disk Space Avail:   690.55 GB / 1022.87 GB (67.5%)
Train Data Rows:    17987
Train Data Columns: 190
Tuning Data Rows:    3673
Tuning Data Columns: 190
Label Column: pv_measurement
Preprocessing data ...
AutoGluon infers your prediction problem is: 'regression' (because dtype of label-column == float and label-values can't be converted to int).
	Label info (max, min, mean, stddev): (999.6, -0.0, 81.53573, 162.39102)
	If 'regression' is not the correct problem_type, please manually specify the problem_type parameter during pred

KeyboardInterrupt: 

In [ ]:
predictor_c.leaderboard(silent=True)

,model,score_val,pred_time_val,fit_time,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,LightGBMXT_BAG_L1,-11.048606,4.467793,738.655129,4.467793,738.655129,1,True,3
1,WeightedEnsemble_L2,-11.048606,4.469773,739.032310,0.001980,0.377180,2,True,6
2,WeightedEnsemble_L3,-11.137366,17.319751,1222.958640,0.000999,0.395211,3,True,11
3,RandomForestMSE_BAG_L2,-11.187752,17.069725,1057.768526,7.095454,120.157869,2,True,9
4,LightGBM_BAG_L2,-11.246028,10.223298,1102.405560,0.249027,164.794903,2,True,8
5,LightGBMXT_BAG_L2,-12.148979,10.218898,979.202121,0.244627,41.591464,2,True,7
6,CatBoost_BAG_L2,-12.681936,10.611483,1083.182326,0.637212,145.571669,2,True,10
7,CatBoost_BAG_L1,-15.520523,0.274693,131.792790,0.274693,131.792790,1,True,5
8,RandomForestMSE_BAG_L1,-17.257606,3.580049,66.714596,3.580049,66.714596,1,True,4
9,KNeighborsDist_BAG_L1,-23.279310,0.808350,0.220083,0.808350,0.220083,1,True,2


In [ ]:
y_pred_c = predictor_c.predict(test_data_c)
y_pred_c=pd.DataFrame(y_pred_c,columns=['pv_measurement'])
y_pred_c #print the DataFrame

,pv_measurement
0,0.100043
1,-0.010812
2,0.465913
3,2.680611
4,22.193726
...,...
715,75.715584
716,42.212669
717,17.481922
718,6.442658


In [ ]:
import os
output_dir = 'autogluon'

if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Save the dataframes into the folder
y_pred_c.to_csv(os.path.join(output_dir, 'y_pred_c.csv'))